In [1]:
import cv2
import numpy as np

In [2]:
'''
#=========================================================================================
说明: 标签防遮挡算法

基本思想: https://stackoverflow.com/questions/3265986/an-algorithm-to-space-out-overlapping-rectangles
         
         输入一组矩形标签
         i = 0
         设置移动步长step(像素长度)

         while 标签内仍存在intersection:
            提取第i个标签
            计算当前所有标签中心点
            计算第i个标签与中心点差值向量 vec
            while 第i个标签仍与其他标签有相交:
                将第i个标签向外移动 vec*step
            i = i + 1

注意事项：
    防止死循环：不能有标签与中心点重合，建议添加重合判断，重合时像随机方向移动
    改良：该算法只管标签不重合，实际上会造成很多标签都挨在一起，建议输入时把矩形标签适当放大，
          算完后缩回去
    更多改良见原网页
               

#=========================================================================================
'''
# Python program to check if rectangles overlap
class Point:
    def __init__(self, x, y = None):

        if y == None:
            self.x = x[0]
            self.y = x[1]
        else:
            self.x = x
            self.y = y

class Rect:
    def __init__(self, p1, p2) -> None:
        self.tl = Point(p1)
        self.br = Point(p2)
        self.c = Point((self.tl.x+ self.br.x)/2, (self.tl.y+ self.br.y)/2)

    def move_away(self, tar, step = 20):
        
        dir = np.array([tar.x - self.c.x, tar.y - self.c.y])
        dir = dir * step/np.sqrt(dir[0]**2 + dir[1]**2)
        dir = np.int32(dir)

        self.tl.x -= dir[0]
        self.tl.y -= dir[1]
        self.br.x -= dir[0]
        self.br.y -= dir[1]
        self.c.x -= dir[0]
        self.c.y -= dir[1]
    
    def intersect(self, rect_ls):
        for rect in rect_ls:
            if overlap(self, rect):
                return True
        return False

def find_center(rect_ls):
    c = Point(0,0)

    if len(rect_ls) == 0:
        return c
    
    for rect in rect_ls:
        c.x += rect.c.x
        c.y += rect.c.y

    c.x = c.x/len(rect_ls)
    c.y = c.y/len(rect_ls)

    return c


def overlap(rect1, rect2):
    l1 = rect1.tl
    l2 = rect2.tl
    r1 = rect1.br
    r2 = rect2.br

    if l1.x == r1.x or l1.y == r1.y or r2.x == l2.x or l2.y == r2.y:
        return False
     
    # If one rectangle is on left side of other
    if l1.x > r2.x or l2.x > r1.x:
        return False
 
    # If one rectangle is above other
    if r1.y > l2.y or r2.y > l1.y:
        return False
 
    return True

def intersect(rect_ls):
    for i in range(len(rect_ls) - 1):
        for j in range(i + 1, len(rect_ls)):
            if overlap(rect_ls[i], rect_ls[j]):
                return True
    return False



In [3]:
im = cv2.imread('bg.jpg')
test_num = 20
rects = []

#初始化测试方块
for i in range(test_num):

    x1 = np.random.randint(200,300)
    x2 = np.random.randint(300,400)
    y1 = np.random.randint(300,400)
    y2 = np.random.randint(200,300)
    rects.append(Rect((x1,y1), (x2, y2)))
    cv2.rectangle(im, (x1,y1), (x2, y2), color = np.random.randint(0,255,3).tolist(), thickness = 3)
    cv2.imwrite('init.png', im)

#deoverlay
iter = 0
while(intersect(rects)):

    center = find_center(rects)
    rect = rects.pop(iter)

    while(rect.intersect(rects)):
        rect.move_away(center)
    rects.insert(0, rect)
    iter += 1

#visualization
im = cv2.imread('bg.jpg')
for rect in rects:
    cv2.rectangle(im, (rect.tl.x,rect.tl.y), (rect.br.x, rect.br.y), color = np.random.randint(0,255,3).tolist(), thickness = 3)
    print(f'x is :{rect.tl.x}, y is:{rect.tl.y}')
cv2.imwrite(f'result.png', im)


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'
